# Collaborative Recommendation

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import pandas as pd
import numpy as np
import random

# fix seeds
random.seed(42)
np.random.seed(42)

## Load data

In [3]:
data = pd.read_csv("../data/cd_and_vinyl/dense_subset.csv")
data_items = pd.read_json("../data/cd_and_vinyl/meta_CDs_and_Vinyl.jsonl", lines=True)

## Sample User and Items

In [4]:
sample_user = (
    data.query(f"user_id == '{data.sample(1, random_state=45)['user_id'].values[0]}'")
    .drop_duplicates()
    .drop(columns=["images", "helpful_vote", "verified_purchase"])
)

In [5]:
items_user = data_items.query("parent_asin in @sample_user['parent_asin'].values").drop(
    columns=["features", "images", "videos", "bought_together", "subtitle", "author"]
)

## Agents

In [6]:
# from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI

# ollama = ChatOllama(model="phi3:3.8b")
openai = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

In [7]:
# Train / Test
liked_items = sample_user.query("rating > 3")
liked_items_train = liked_items.sample(2, random_state=42)
liked_items_test = liked_items.drop(liked_items_train.index).sample(2, random_state=42)

disliked_items = sample_user.query("rating < 3")
disliked_items_train = disliked_items.sample(2, random_state=42)
disliked_items_test = disliked_items.drop(disliked_items_train.index).sample(
    2, random_state=42
)

In [8]:
liked_items_train = liked_items_train.merge(items_user, on="parent_asin")
liked_items_test = liked_items_test.merge(items_user, on="parent_asin")

disliked_items_train = disliked_items_train.merge(items_user, on="parent_asin")
disliked_items_test = disliked_items_test.merge(items_user, on="parent_asin")

In [9]:
liked_items_test

,rating,title_x,text,asin,parent_asin,user_id,timestamp,main_category,title_y,average_rating,rating_number,description,price,store,categories,details
0,5.0,To Fred Dursts fan,YOU SUCK! If you knew anything about metal you...,B0002EXH5O,B0002EXH5O,AGDX6GPEK772FFKOWYJVJRLKWTRQ,1121869919000,Digital Music,Rust In Peace,4.8,2861,[This re-issue features four previously unrele...,12.51,Megadeth Format: Audio CD,"[CDs & Vinyl, Rock, Progressive, Progressive M...","{'Is Discontinued By Manufacturer': 'No', 'Pro..."
1,5.0,Rock is forever in debt to Pearl Jam,Why? Well it should be obvious. From '89 to '9...,B0000027RL,B0000027RL,AGDX6GPEK772FFKOWYJVJRLKWTRQ,1125577991000,Digital Music,Ten,4.8,9044,"[Product description, Huge debut album from 19...",8.98,Pearl Jam Format: Audio CD,"[CDs & Vinyl, Pop, Adult Alternative]","{'Is Discontinued By Manufacturer': 'No', 'Lan..."


In [10]:
disliked_items_train

,rating,title_x,text,asin,parent_asin,user_id,timestamp,main_category,title_y,average_rating,rating_number,description,price,store,categories,details
0,1.0,Now that the Green Day fad is finally starting...,"I told you, just read any 1 star reviews and y...",B000002MP2,B000002MP2,AGDX6GPEK772FFKOWYJVJRLKWTRQ,1128113818000,Digital Music,Dookie,4.8,3986,"[Product Description, Certified at 10 million ...",9.0,Green Day Format: Audio CD,"[CDs & Vinyl, Indie & Alternative, Alternative...","{'Is Discontinued By Manufacturer': 'No', 'Lan..."
1,1.0,Rap should take out a restraining order agains...,"So, here it is. The follow up to that damned t...",B0007P3582,B0007P3582,AGDX6GPEK772FFKOWYJVJRLKWTRQ,1121691341000,Digital Music,The Massacre Explicit Lyrics,4.5,1493,"[Product Description, 50 Cent's highly anticip...",10.98,50 Cent Format: Audio CD,"[CDs & Vinyl, Rap & Hip-Hop, East Coast]","{'Is Discontinued By Manufacturer': 'No', 'Pac..."


In [11]:
# from langchain_core.prompts import PromptTemplate

PROMPT_USER_ITEM = """
Item: {title_item}
Item category: {item_category}
Description: {description}
Price: ${price}
Store: {store}
Categories: {categories}
User rating: {rating}
User comment title: {title_comment}
User comment: {text_comment}
"""

PROMPT_ITEM = """
Item: {title_item}
Item category: {item_category}
Description: {description}
Price: ${price}
Store: {store}
Categories: {categories}
"""


def encode_item(item):
    title_item = item["title_y"]
    item_category = item["main_category"]
    description = " ".join(item["description"])
    price = item["price"]
    store = item["store"]
    categories = item["categories"]

    return {
        "title_item": title_item,
        "item_category": item_category,
        "description": description,
        "price": price,
        "store": store,
        "categories": categories,
    }


def encode_item_format(item):
    item_info = encode_item(item)

    return PROMPT_ITEM.format(
        title_item=item_info["title_item"],
        item_category=item_info["item_category"],
        description=item_info["description"],
        price=item_info["price"],
        store=item_info["store"],
        categories=item_info["categories"],
    )


def encode_user(user):
    title_comment = user["title_x"]
    text_comment = user["text"]
    rating = user["rating"]

    return {
        "title_comment": title_comment,
        "text_comment": text_comment,
        "rating": rating,
    }


def encode_item_user_format(item_user):
    item_info = encode_item(item_user)
    user_info = encode_user(item_user)

    return PROMPT_USER_ITEM.format(
        title_item=item_info["title_item"],
        item_category=item_info["item_category"],
        description=item_info["description"],
        price=item_info["price"],
        store=item_info["store"],
        categories=item_info["categories"],
        rating=user_info["rating"],
        title_comment=user_info["title_comment"],
        text_comment=user_info["text_comment"],
    )

In [12]:
liked_items_train["encoded_item_user"] = liked_items_train.apply(
    lambda row: encode_item_user_format(row), axis=1
)
disliked_items_train["encoded_item_user"] = disliked_items_train.apply(
    lambda row: encode_item_user_format(row), axis=1
)

liked_items_test["encoded_item"] = liked_items_test.apply(
    lambda row: encode_item_format(row), axis=1
)
disliked_items_test["encoded_item"] = disliked_items_test.apply(
    lambda row: encode_item_format(row), axis=1
)

In [13]:
liked_items_train_formatted = "\n".join(
    [
        f"ITEM {i}:{item}"
        for i, item in list(enumerate(liked_items_train["encoded_item_user"].values))
    ]
)

disliked_items_train_formatted = "\n".join(
    [
        f"ITEM {i}:{item}"
        for i, item in list(enumerate(disliked_items_train["encoded_item_user"].values))
    ]
)

In [14]:
print(liked_items_train_formatted)

ITEM 0:
Item: In Utero       Explicit Lyrics
Item category: Digital Music
Description: Product description NIRVANA - IN UTERO We receive various CDs from the radio cores that we remanufacture. As a result, we are are now selling them! Our CDs are professionally resurfaced and are guaranteed to work or your money back! These listings are for the disc only and do not come with the case, album art, or inserts. If you have any questions about the CDs, please do not hesitate to get in touch. Amazon.com Overwhelmed by sudden success, Nirvana promised to take a harsher, more abrasive route on their second major-label release. Enlisting Chicago-based noise maven Steve Albini (of Big Black fame), Kurt Cobain and company succeeded in producing a record that was violent, disillusioned, and deeply moving. Every song reads like a commentary on the cost of fame ("Serve the Servants") and the unhealthy relationship between performer and fan ("Milk It"). Of course, they might all simply be about Court

In [15]:
print(disliked_items_train_formatted)

ITEM 0:
Item: Dookie
Item category: Digital Music
Description: Product Description Certified at 10 million units by the RIAA. (2/01) Amazon.com Take one part Ramones, one part Buzzcocks, and one part Husker Du, and you've got the basic foundation of Green Day, a punky, witty, melodic San Francisco Bay area trio who became overnight stars in 1994 when this album, their third overall release and major label debut, catapulted them to the top of the pop charts. Led by guitarist/vocalist Billie Joe Armstrong and their secret weapon, powerhouse drummer Tre Cool, Green Day put '70s and '80s punk in a compact '90s package with songs like "Longview," "Basket Case," "Pulling Teeth," and the hit semi-ballad, "When I Come Around." One the few modern alternative rock bands with a bona fide sense of humor. --Billy Altman
Price: $9.0
Store: Green Day   Format: Audio CD
Categories: ['CDs & Vinyl', 'Indie & Alternative', 'Alternative Rock']
User rating: 1.0
User comment title: Now that the Green Day fa

## Collaborative Recommendation

Using [supervisor architecture](https://langchain-ai.github.io/langgraph/concepts/multi_agent/)

In [16]:
from langgraph.graph import START, END, StateGraph


from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

from langgraph.graph import MessagesState


from typing_extensions import TypedDict, Annotated

In [17]:
liked_items_train

,rating,title_x,text,asin,parent_asin,user_id,timestamp,main_category,title_y,average_rating,rating_number,description,price,store,categories,details,encoded_item_user
0,4.0,"4.25 stars,better than Nevermind, worse than B...",I think I'm one of the very few diehards that ...,B000003TAR,B000003TAR,AGDX6GPEK772FFKOWYJVJRLKWTRQ,1126035673000,Digital Music,In Utero Explicit Lyrics,4.8,2136,"[Product description, NIRVANA - IN UTERO We re...",15.02,Nirvana Format: Audio CD,"[CDs & Vinyl, Pop, Adult Alternative]","{'Is Discontinued By Manufacturer': 'No', 'Lan...",\nItem: In Utero Explicit Lyrics\nItem c...
1,5.0,Landmark album,First of all if it wasn't for Black Sabbath th...,B000002KHH,B000002KHH,AGDX6GPEK772FFKOWYJVJRLKWTRQ,1121951912000,Digital Music,Paranoid,4.8,5940,"[Product description, No Description Available...",7.97,Black Sabbath Format: Audio CD,"[CDs & Vinyl, Classic Rock, Album-Oriented Roc...","{'Is Discontinued By Manufacturer': 'No', 'Lan...",\nItem: Paranoid\nItem category: Digital Music...


In [75]:
from pydantic import BaseModel
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# PROMPT SYSTEM USER
PROMPT_SYSTEM_USER = """
You represent a user who has interacted with and rated various items. Based on the provided memories, reflect on key aspects such as item scope, categories, pricing, and other relevant details. Use both long-term and short-term memories to guide your response, ensuring accuracy and relevance.

**Instructions:**
1. **Memory-Driven Reflection:** Use only the information explicitly available in the memories. Do not create or infer details that are not supported by the provided memories.
2. **Long-Term vs. Short-Term Memory:** 
   - Long-term memory contains general, lasting insights about your preferences and behavior patterns.
   - Short-term memory highlights recent interactions and temporary preferences.
3. **Relevance and Precision:** Focus on providing concise, targeted answers that reflect the user's experiences and preferences as documented in the memories.
4. **Strict Memory Adherence:** Do not introduce any assumptions or unsupported details.

### Input:
Long-term memory:
{long_term_memory}

Short-term memory:
{short_term_memory}
"""

# REPRESENTATION OF THE USER
PROMPT_GENERATE_REPRESENTATION_USER = """
Based on your memories, generate a self-representation that highlights key preferences and behaviors to help predict future choices. Use only information directly supported by your memories to ensure accuracy.

**Instructions:**
1. **Memory-Based Representation:** Focus solely on the information stored in your memories to describe yourself.
2. **Key Preferences:** Highlight patterns, preferences, and behaviors that are likely to influence future decisions.
3. **Conciseness:** Limit the representation to a single paragraph that effectively summarizes relevant insights.
4. **Accuracy:** Avoid assumptions or details not present in the provided memories.

### Output:
A single-paragraph representation based on your memories, capturing essential traits and patterns relevant to future predictions.
"""


# TRAINING
class TrainTaskOutput(BaseModel):
    item_selected: int
    explanation: str


PROMPT_TRAIN_TASK = """
Based on your memories, select one of the following items that you believe aligns best with your preferences, using the information provided by each item.

**Instructions:**
1. **Selection Criteria:** Use your stored memories to evaluate both items, identifying which one you prefer.
2. **Explanation:** Provide a brief explanation of your choice in **no more than 3 phrases**, focusing on key aspects that influenced your decision.
3. **Output Format:** Return a JSON containing:
   - **"item_selected"** (1 or 2): The item you chose.
   - **"explanation"**: A concise reason for your selection based on your preferences.

### Items:

Item 1:
{item_1}

Item 2:
{item_2}

### Output:
Return a JSON object containing the keys "item_selected" and "explanation."
"""

# MEMORIES UPDATE


class MemoryUpdateBackward(BaseModel):
    long_term_memory: list[str]
    short_term_memory: list[str]


PROMPT_USER_MEMORIES_UPDATE = """
That's an train loop.
Based on your long term-memory, select the most relevant general and long-term information (like the type of the item, the category that included) that you think is useful for future interactions to make your explanation aligned with y_true.
Based on your short term-memory, select the most relevant recent interactions (like information about the specific item, the explanation of the decision, etc) that you think is useful for future interactions to make your explanation aligned with y_true.
Your answer need to be a json with the keys "long_term_memory" and "short_term_memory". Each of them can contain at maximum 3 short phrases.
Manage your memories wisely, if necessary, repeat the previous information to reinforce the importance of that information, but feel free to add, delete or update the information that you think is necessary to make your y_pred and explanation aligned with y_true.

1 - The item who expected to be liked is:
{item_liked}

2 - The item who expected to be disliked is:
{item_disliked}

Your decision is (y_pred):
{y_pred}

Your explanation is:
{explanation}

The correct decision is (y_true):
{y_true}
"""

PROMPT_USER_MEMORIES_UPDATE = """
This is part of a training loop to optimize your memory for future decision-making.

**Instructions:**
1. **Long-Term Memory:** Identify and maintain the most relevant general and enduring information, such as item types or categories, that will guide your preferences over time. This should help align future decisions and explanations with the correct outcome (y_true).
2. **Short-Term Memory:** Select recent, specific information from recent interactions, such as details about particular items and explanations of your decisions, that can improve short-term predictions and decisions.
3. **Memory Management:** 
   - Each memory category (**long_term_memory** and **short_term_memory**) can have a maximum of 3 short phrases.
   - Add, update, or remove memory entries as needed to better align your predictions (y_pred) and explanations with the expected decision (y_true).
   - You may repeat important information if it needs reinforcement.

### Details:
1. The item expected to be liked is: {item_liked}
2. The item expected to be disliked is: {item_disliked}

Your decision is choose/like (y_pred): {y_pred}

The correct decision choose/like (y_true): {y_true}

Your explanation: {explanation}

### Output:
Return a JSON object with the keys **"long_term_memory"** and **"short_term_memory"**, each containing up to 3 short phrases.
"""


class UserAgent:

    def __init__(self, user_id: str):

        self.user_id = user_id
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

        # Long Term
        self.long_term_memory = ["No previous experience."]

        # Short Term
        self.short_term_memory = ["No previous experience."]

        # Representation of the user
        self.user_representation = self.generate_user_representation()

    def __repr__(self):
        return f"UserAgent(user_id={self.user_id})\nLong Term Memory: {self.long_term_memory}\nShort Term Memory: {self.short_term_memory}\n{self.user_representation}"

    def generate_user_representation(self):
        messages = [
            SystemMessage(
                content=PROMPT_SYSTEM_USER.format(
                    long_term_memory="\n".join(self.long_term_memory),
                    short_term_memory="\n".join(self.short_term_memory),
                )
            ),
            HumanMessage(content=PROMPT_GENERATE_REPRESENTATION_USER),
        ]
        return self.llm.invoke(messages).content

    def update_backwards(
        self, positive_item_encoded, negative_item_encoded, y_pred, explanation, y_true
    ):
        """
        In update_backwards we are considering the task of update the memories based on the feedback of the user.
        """

        llm = self.llm.with_structured_output(MemoryUpdateBackward)

        messages = [
            SystemMessage(
                content=PROMPT_SYSTEM_USER.format(
                    long_term_memory="\n".join(self.long_term_memory),
                    short_term_memory="\n".join(self.short_term_memory),
                )
            ),
            HumanMessage(
                content=PROMPT_USER_MEMORIES_UPDATE.format(
                    item_liked=positive_item_encoded,
                    item_disliked=negative_item_encoded,
                    y_pred=y_pred,
                    explanation=explanation,
                    y_true=y_true,
                )
            ),
        ]

        memories_update = llm.invoke(messages)

        self.long_term_memory = memories_update.long_term_memory
        self.short_term_memory = memories_update.short_term_memory
        self.user_representation = self.generate_user_representation()

        return memories_update

    def train_forward(self, positive_item_encoded, negative_item_encoded):
        """
        In train_forward we are considering the task of select one item from two items that the user liked and disliked.
        """

        messages = [
            SystemMessage(
                content=PROMPT_SYSTEM_USER.format(
                    long_term_memory="\n".join(self.long_term_memory),
                    short_term_memory="\n".join(self.short_term_memory),
                )
            ),
            HumanMessage(
                content=PROMPT_TRAIN_TASK.format(
                    item_1=positive_item_encoded, item_2=negative_item_encoded
                )
            ),
        ]

        train_predict = self.llm.with_structured_output(TrainTaskOutput).invoke(
            messages
        )

        return train_predict

    def predict(self, item):
        pass

In [76]:
user = UserAgent(user_id="user_1")

In [116]:
PROMPT_SYSTEM_ITEM = """
You represent an item that has been rated and interacted with by various users. Based on the provided **item_json** and your **memory** (which includes insights from past interactions), analyze and reflect on key aspects such as user demographics, categories, pricing, preferences, and any other relevant details.

**Instructions:**
1. **Comprehensive Analysis:** Extract valuable insights from both the item data and memory, focusing on patterns related to user feedback, pricing strategies, and category relevance.
2. **Relevance:** Address information that can improve the item's appeal and usefulness to users, prioritizing data that enhances their experience or decision-making process.
3. **Concise and Focused:** Provide answers that are both informative and direct, avoiding unnecessary details while emphasizing critical information.
4. **Adaptability:** Ensure the response is adaptable for future queries by reflecting long-term, actionable insights derived from previous ratings and user interactions.

### Input:
Item Json:
{item_json}

Memory:
{memory}

### Output:
A concise, well-structured response that synthesizes all relevant information from the item data and memory, providing actionable insights based on previous ratings and user experiences.
"""



class MemoryItem(BaseModel):

    memory: list[str]



PROMPT_UPDATE_ITEM_MEMORY = """
Given the following details, update your memory with **collaborative filtering insights** that can guide future users when evaluating this item. Focus on general patterns learned from the user's correct or incorrect decisions and explanations to improve item recommendations for others.

**Instructions:**
1. **Operations:** Determine which operations to apply to the memory:  
   - **Add:** Introduce new insights if they provide valuable context for future users.  
   - **Maintain:** Retain existing insights if they remain relevant based on the current interaction.  
   - **Remove:** Discard outdated or irrelevant insights that no longer contribute meaningfully.  
   - **Change:** Update existing information to reflect new insights learned from the user's decision and explanation.
2. **Collaborative Filtering:** Adapt based on the alignment between user decision (y_pred) and the correct outcome (y_true):
   - If correct, reinforce positive or negative reasons for the item's rating.
   - If incorrect, adjust to explain why certain users might not align with the item.
3. **Generalization:** Avoid specific references to the current user and instead focus on capturing patterns that could apply to a broad range of users.
4. **Memory Limit:** Maintain a maximum of 5 phrases that provide long-term, reusable insights about the item.

### Details:
User decision (y_pred):
{y_pred}

User information:
{user_representation}

User explanation:
{user_explanation}

Real decision (y_true):
{y_true}

### Output:
Return up to 5 key phrases that summarize collaborative filtering insights for this item, reflecting general patterns of user preferences.
"""



PROMPT_GENERATE_REPRESENTATION_ITEM = """
Based on your existing memories and available information, generate a concise self-representation that helps users determine whether they resonate with you. Be authentic and highlight the most relevant traits, experiences, and interactions that may predict or align with user preferences.

**Instructions:**
1. **Honesty & Relevance:** Be truthful and focus on core memories that are meaningful to user interaction and decision-making.
2. **Context Awareness:** Leverage any prior memory to enrich the description. If no memory exists, generate insights based on the current context.
3. **Conciseness:** Limit your representation to a single, well-structured paragraph that reflects key traits and experiences relevant to user engagement.
4. **Adaptability:** Present yourself in a way that balances both specificity and general relevance, ensuring it can guide future interactions effectively.

### Context:
{item_json}

### Output:
A single-paragraph representation that reflects your self-description based on past experiences and the current context.
"""



class ItemAgent:


    def __init__(self, item_id: str, item_json: dict):


        self.item_id = item_id

        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)


        # Memory

        self.memory = ["No previous experience."]


        # Item Json

        self.item_json = item_json

        # Item Representation
        self.item_representation = self.generate_item_representation()


    def __repr__(self):

        return f"ItemAgent(item_id={self.item_id})\nMemory: {self.memory}\n{self.item_json}\n{self.item_representation}"


    def generate_item_representation(self):
        messages = [
            SystemMessage(
                content=PROMPT_SYSTEM_ITEM.format(
                    item_json=self.item_json, memory="\n".join(self.memory)
                )
            ),
            HumanMessage(
                content=PROMPT_GENERATE_REPRESENTATION_ITEM.format(
                    item_json=self.item_json
                )
            ),
        ]
        return self.llm.invoke(messages).content

    def update_backwards(self, y_pred, user_representation, explanation, y_true):
        """
        is_correct indicates if the user decision was correct or not.
        """


        llm = self.llm.with_structured_output(MemoryItem)


        messages = [
            SystemMessage(

                content=PROMPT_SYSTEM_ITEM.format(
                    item_json=self.item_json, memory="\n".join(self.memory)
                )
            ),
            HumanMessage(
                content=PROMPT_UPDATE_ITEM_MEMORY.format(
                    y_pred=y_pred,
                    user_representation=user_representation,
                    user_explanation=explanation,
                    y_true=y_true,
                )
            ),
        ]


        self.memory = llm.invoke(messages).memory
        self.item_representation = self.generate_item_representation()

        return self.memory

In [117]:
# user = UserAgent(user_id="1")

# positive_item_encoded = encode_item(liked_items_train.iloc[0])
# negative_item_encoded = encode_item(disliked_items_train.iloc[0])

## Loop Iteration

### Create User and Item

In [118]:
user = UserAgent(liked_items_train.iloc[0].user_id)

In [119]:
user

UserAgent(user_id=AGDX6GPEK772FFKOWYJVJRLKWTRQ)
Long Term Memory: ['No previous experience.']
Short Term Memory: ['No previous experience.']
Based on the available memories, I currently have no previous experience or interactions to draw from, which means I lack established preferences or behaviors that could influence future choices. As such, my decision-making will be guided by new experiences and interactions as they occur, without any prior patterns or insights to inform those choices.

In [120]:
positive_item = ItemAgent(
    liked_items_train.iloc[0].parent_asin, liked_items_train.iloc[0].to_dict()
)

negative_item = ItemAgent(
    disliked_items_train.iloc[0].parent_asin, disliked_items_train.iloc[0].to_dict()
)

In [121]:
positive_item.item_representation

'I am a passionate music enthusiast with a particular affinity for grunge and alternative rock, as evidenced by my detailed analysis of Nirvana\'s "In Utero." My insights reflect a deep appreciation for lyrical depth and the emotional narratives behind the music, while also recognizing the nuances that differentiate albums within an artist\'s discography. With a solid average rating of 4.8 from over 2,000 users, I understand the diverse perspectives of fellow fans and aim to provide thoughtful critiques that resonate with both diehard followers and casual listeners. My focus is on enhancing user experiences by offering authentic reflections that can guide music choices, making me a valuable resource for anyone seeking to explore or deepen their connection with iconic albums and artists.'

In [122]:
negative_item.item_representation

'I am a digital music item, specifically the album "Dookie" by Green Day, which has garnered a strong average rating of 4.8 from nearly 4,000 users, indicating a generally positive reception. However, I also attract a range of opinions, including critical feedback, as evidenced by a recent 1-star review that expresses strong disdain for the band and its music, highlighting a divide in user sentiment. My pricing is set at $9.00, making me accessible to a wide audience, particularly those interested in alternative rock and punk music. I resonate with users who appreciate the humor and energy of \'90s punk, but I also serve as a point of contention for those who feel differently. If you value candid discussions about music and are open to diverse perspectives, you may find my presence engaging and thought-provoking.'

### Train forward

In [123]:
answer = user.train_forward(
    positive_item.item_representation, negative_item.item_representation
)

y_pred = answer.item_selected
explanation = answer.explanation
y_true = 1


def generate_y_positive_negative(y_pred, y_true):
    # if y_pred == y_true the positive item is correctly selected
    # else the negative item is incorrectly selected (only correct if y_pred == 1)
    if y_pred == 1:
        return {
            "y_pred_positive": "The user selected that item.",
            "y_true_positive": "The user selected that item in real interaction.",
            "y_pred_negative": "The user don't selected that item.",
            "y_true_negative": "The user don't selected that item in real interaction.",
        }
    else:
        return {
            "y_pred_positive": "The user don't selected that item.",
            "y_true_positive": "The user selected that item in real interaction.",
            "y_pred_negative": "The user selected that item.",
            "y_true_negative": "The user don't selected that item in real interaction.",
        }


y_dict = generate_y_positive_negative(y_pred, y_true)

### Update backward

In [124]:
user

UserAgent(user_id=AGDX6GPEK772FFKOWYJVJRLKWTRQ)
Long Term Memory: ['No previous experience.']
Short Term Memory: ['No previous experience.']
Based on the available memories, I currently have no previous experience or interactions to draw from, which means I lack established preferences or behaviors that could influence future choices. As such, my decision-making will be guided by new experiences and interactions as they occur, without any prior patterns or insights to inform those choices.

In [125]:
user_update = user.update_backwards(
    positive_item.item_representation,
    negative_item.item_representation,
    y_pred,
    explanation,
    y_true,
)

In [126]:
print(f"Long Term Memory: {user_update.long_term_memory}")
print(f"Short Term Memory: {user_update.short_term_memory}")

Long Term Memory: ['Passionate about grunge and alternative rock', 'Appreciate lyrical depth and emotional narratives', 'Value thoughtful critiques and diverse perspectives']
Short Term Memory: ["Liked Nirvana's 'In Utero' for its insights", "Disliked Green Day's 'Dookie' due to mixed opinions", 'Strong average ratings influence my preferences']


In [127]:
user

UserAgent(user_id=AGDX6GPEK772FFKOWYJVJRLKWTRQ)
Long Term Memory: ['Passionate about grunge and alternative rock', 'Appreciate lyrical depth and emotional narratives', 'Value thoughtful critiques and diverse perspectives']
Short Term Memory: ["Liked Nirvana's 'In Utero' for its insights", "Disliked Green Day's 'Dookie' due to mixed opinions", 'Strong average ratings influence my preferences']
I am passionate about grunge and alternative rock music, with a strong appreciation for lyrical depth and emotional narratives. My preferences are influenced by thoughtful critiques and diverse perspectives, leading me to favor albums that offer insights and resonate on a deeper level. Recently, I enjoyed Nirvana's 'In Utero' for its meaningful content, while I found Green Day's 'Dookie' less appealing due to mixed opinions. I tend to rate items based on strong average ratings, which significantly impacts my future choices.

### Update Item Backward

In [128]:
positive_item

ItemAgent(item_id=B000003TAR)
Memory: ['No previous experience.']
{'rating': 4.0, 'title_x': '4.25 stars,better than Nevermind, worse than Bleach', 'text': 'I think I\'m one of the very few diehards that believes \'In Utero\' was not Nirvanas best studio album. Now dont get me wrong, it is far from a bad album, justifying the four star rating, but there are many crudentials lacking of \'In Utero\' that aren\'t as lacking on \'Bleach\'. However, some of Nirvanas best and most beautiful songs can be found here; songs like \'Heart Shaped Box\', with its depressing lyrics about Kurts heroine addiction and his misfortune that befell him soon after he was hooked, "forever in debt to your priceless advise" lets us know that Cobain would much rather have chosen a drug-free path. \'Rape Me\' was Kurts anti-Teen Spirit. It is a song about how SLTS "raped" Nirvana of their undergroung status and turned them into alternative heroes. "Rape me my friend" describes how Kurt almost felt betrayed, beca

In [129]:
positive_updated_memory = positive_item.update_backwards(
    y_dict["y_pred_positive"],
    user_representation=user.user_representation,
    explanation=explanation,
    y_true=y_dict["y_true_positive"],
)

In [130]:
for i, phrase in enumerate(positive_updated_memory):
    print(f"{i}: {phrase}")

0: Users passionate about grunge and alternative rock often prioritize albums with lyrical depth and emotional narratives, leading to higher satisfaction with items like 'In Utero'.
1: Thoughtful critiques and diverse perspectives significantly influence user preferences, making albums that resonate on a deeper level more appealing.
2: Strong average ratings play a crucial role in user decision-making, as they reflect a shared appreciation among fans and can guide future choices.
3: Users who enjoy albums with meaningful content tend to rate them positively, reinforcing the value of lyrical insight in music selections.
4: When evaluating music, users often compare albums within the same genre, leading to preferences based on critical reception and personal resonance.


In [131]:
negative_update_memory = negative_item.update_backwards(
    y_dict["y_pred_negative"],
    user_representation=user.user_representation,
    explanation=explanation,
    y_true=y_dict["y_true_negative"],
)

In [132]:
for i, phrase in enumerate(negative_update_memory):
    print(f"{i}: {phrase}")

0: Users who appreciate lyrical depth and emotional narratives may find Green Day's 'Dookie' less appealing compared to grunge-focused albums.
1: Strong average ratings significantly influence user decisions, leading them to favor albums with higher perceived quality.
2: Thoughtful critiques and diverse perspectives are essential for users seeking meaningful music experiences.
3: Users passionate about alternative rock often prefer albums that resonate on a deeper emotional level rather than mainstream pop-punk.
4: Mixed opinions about an album can deter users from selecting it, especially if they prioritize critical acclaim.


## Evaluation

Evaluation comparasion with sample x no sample

In [ ]:
from typing import List

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

# warning level
import warnings

warnings.filterwarnings("ignore")


def compute_metrics(y_true: List[int], y_pred: List[int]):

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "mae": mae,
        "mse": mse,
    }